# IMDB Sentiment Analysis

The data is split evenly with 25k reviews intended for training and 25k for testing your classifier. Moreover, each set has 12.5k positive and 12.5k negative reviews.

IMDb lets users rate movies on a scale from 1 to 10. To label these reviews the curator of the data labeled anything with ≤ 4 stars as negative and anything with ≥ 7 stars as positive. Reviews with 5 or 6 stars were left out.

**Import the required libraries**

In [28]:
import numpy as np
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings("ignore")

**Load Data**

In [2]:
reviews_train = []
for line in open('data/full_train.txt', 'r', encoding='latin1'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('data/full_test.txt', 'r', encoding='latin1'):
    
    reviews_test.append(line.strip())

**See one of the elements in the list**

In [4]:
print(len(reviews_train))
print(len(reviews_test))

reviews_train[:2]

25000
25000


['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!',
 'Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything fro

The raw text is pretty messy for these reviews so before we can do any analytics we need to clean things up


**Use Regular expressions to remove the non text characters, and the html tags**

In [5]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [6]:
reviews_train_clean[5]

"this isn't the comedic robin williams nor is it the quirky insane robin williams of recent thriller fame this is a hybrid of the classic drama without over dramatization mixed with robin's new love of the thriller but this isn't a thriller per se this is more a mystery suspense vehicle through which williams attempts to locate a sick boy and his keeper also starring sandra oh and rory culkin this suspense drama plays pretty much like a news report until william's character gets close to achieving his goal i must say that i was highly entertained though this movie fails to teach guide inspect or amuse it felt more like i was watching a guy williams as he was actually performing the actions from a third person perspective in other words it felt real and i was able to subscribe to the premise of the story all in all it's worth a watch though it's definitely not friday saturday night fare it rates a   from the fiend "

# Vectorization
In order for this data to make sense to our machine learning algorithm we’ll need to convert each review to a numeric representation, which we call vectorization.

The simplest form of this is to create one very large matrix with one column for every unique word in your corpus (where the corpus is all 50k reviews in our case). Then we transform each review into one row containing 0s and 1s, where 1 means that the word in the corpus corresponding to that column appears in that review. That being said, each row of the matrix will be very sparse (mostly zeros). This process is also known as one hot encoding. Use the *CountVectorizer* method.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(reviews_train_clean)

X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [12]:
print(X_baseline.shape)
baseline_vectorizer.vocabulary_
X_baseline

(25000, 87063)


<25000x87063 sparse matrix of type '<class 'numpy.int64'>'
	with 3410713 stored elements in Compressed Sparse Row format>

In [13]:
vectorizer_c = CountVectorizer()
vectorizer_c.fit(reviews_train_clean)

X_baseline_c = vectorizer_c.transform(reviews_train_clean)
print(X_baseline.shape)

(25000, 87063)


In [14]:
print(vectorizer_c.get_feature_names())

man', 'wolfpack', 'wolfs', 'wolfstein', 'wolfy', 'wolhiem', 'wolitzer', 'wolliaston', 'wollter', 'wolske', 'wolskewho', 'wolsky', 'wolverine', 'wolverineis', 'wolverines', 'wolverinethen', 'wolverinish', 'wolves', 'wolvie', 'womack', 'woman', 'womancampbell', 'womanfor', 'womanhood', 'womani', 'womaniser', 'womanises', 'womanising', 'womanize', 'womanizer', 'womanizing', 'womanjoki', 'womankate', 'womanly', 'womanmost', 'womanness', 'womano', 'womanplayed', 'womans', 'womanthe', 'womanwatch', 'womanwealth', 'womanwho', 'womanâ', 'womb', 'wombat', 'women', 'womenactuallyhe', 'womenalsothe', 'womenbach', 'womenfolk', 'womenrespectively', 'womens', 'womensarah', 'womenthe', 'womenthen', 'womenthey', 'womenthis', 'womenwho', 'womenâ', 'womman', 'won', 'wonand', 'wonder', 'wonderat', 'wonderbooki', 'wondered', 'wonderedcould', 'wonderfalls', 'wonderful', 'wonderfuli', 'wonderfulit', 'wonderfull', 'wonderfully', 'wonderfullyas', 'wonderfullygary', 'wonderfullythat', 'wonderfulthe', 'wonderfu

# Train a Baseline Model

Train a Logistic Regression model after transforming the data with CountVectorized

* They’re easy to interpret
* Linear models tend to perform well on sparse datasets like this one
* They learn very fast compared to other algorithms.

Test models with C values of [0.01, 0.05, 0.25, 0.5, 1] and see wich is the best value for C, and calculate the accuracy

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

target = [1 if i < 12500 else 0 for i in range(25000)]

def train_model(X_TRAIN, X_TEST):
    lr = LogisticRegression()

    params = {
        'C': [0.01, 0.05, 1.25, 0.5, 1]
    }

    grid = GridSearchCV(lr, params, cv=5)
    grid.fit(X_TRAIN, target)

    print('Final accuracy: %s' % accuracy_score(target, grid.best_estimator_.predict(X_TEST)))

In [29]:
train_model(X_baseline, X_test_baseline)

Final accuracy: 0.88188


# Remove Stop Words

Stop words are the very common words like ‘if’, ‘but’, ‘we’, ‘he’, ‘she’, and ‘they’. We can usually remove these words without changing the semantics of a text and doing so often (but not always) improves the performance of a model. Removing these stop words becomes a lot more useful when we start using longer word sequences as model features (see n-grams below).

Before we apply the CountVectorized, lets remove the stopwords, included in nltk.corpus

Then apply the CountVectorizer, and train the Logistic regression model and obtain the accuracy.

In [19]:
# !pip install nltk
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\artur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords

stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [22]:
stopwords.words('spanish')[:30]

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque']

In [23]:
print(len(stopwords.words('english')))
print(len(stopwords.words('spanish')))

179
313


In [24]:
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        
        # Para cada review elimina las stopwords, y separa todas las palabras por espacio
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [26]:
print(reviews_train[0])
print()
print(reviews_train_clean[0])
print()
print(no_stop_words_train[0])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!

bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my  years in the teaching profession lead me to believe that bromwell high's

In [30]:
cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)

X = cv.transform(no_stop_words_train)

X_test = cv.transform(no_stop_words_test)

train_model(X, X_test)

Final accuracy: 0.87936


In [31]:
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 87046)
Stop words eliminadas: 17


In [ ]:
cv = CountVectorizer(binary=True, stop_words=english_stop_words)

cv.fit(reviews_train_clean)

X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

train_model(X, X_test)

In [ ]:
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

**Note:** In practice, an easier way to remove stop words is to just use the stop_words argument with any of scikit-learn’s ‘Vectorizer’ classes. If you want to use NLTK’s full list of stop words you can do stop_words='english’. In practice I’ve found that using NLTK’s list actually decreases my performance because its too expansive, so I usually supply my own list of words. For example, stop_words=['in','of','at','a','the'] .

A common next step in text preprocessing is to normalize the words in your corpus by trying to convert all of the different forms of a given word into one. Two methods that exist for this are Stemming and Lemmatization.

# Stemming

Stemming is considered to be the more crude/brute-force approach to normalization (although this doesn’t necessarily mean that it will perform worse). There’s several algorithms, but in general they all use basic rules to chop off the ends of words.

NLTK has several stemming algorithm implementations. We’ll use the Porter stemmer. Most used:
* PorterStemmer
* SnowballStemmer

Apply a PoterStemmer, vectorize, and train the model again

In [33]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']

singles = [stemmer.stem(i) for i in plurals]

print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [34]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
            
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [35]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('spanish')

plurals = ['corriendo', 'casas', 'playa', 'volando', 'volar', 'volveré']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

corr cas play vol vol volv


In [36]:
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()

    output = [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

    return output

stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(stemmed_reviews_train)
X_stem = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

train_model(X_stem, X_test)

Final accuracy: 0.87736


In [37]:
print(X_baseline.shape)
print(X_stem.shape)
print("Diff X normal y X tras stemmer y vectorización:", X_baseline.shape[1] - X_stem.shape[1])

(25000, 87063)
(25000, 66852)
Diff X normal y X tras stemmer y vectorización: 20211


# Lemmatization

Lemmatization works by identifying the part-of-speech of a given word and then applying more complex rules to transform the word into its true root.

In [39]:
'''
Para lematizacion en español: pip install es-lemmatizer
'''
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\artur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

plurals = ['caresses', 'flies', 'dies', 'mules', 'studies',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
            
singles = [lemmatizer.lemmatize(plural) for plural in plurals]

print(' '.join(singles))

caress fly dy mule study died agreed owned humbled sized meeting stating siezing itemization sensational traditional reference colonizer plotted


In [41]:
def get_lemmatized_text(corpus):
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

# Lematizamos las reviews
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

# Vectorizamos con conteo tras lematizar
cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)
X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

train_model(X, X_test)

Final accuracy: 0.87956


In [42]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 80326)
Diff X normal y X tras lematizador y vectorización: 6737


# n-grams

We can potentially add more predictive power to our model by adding two or three word sequences (bigrams or trigrams) as well. For example, if a review had the three word sequence “didn’t love movie” we would only consider these words individually with a unigram-only model and probably not capture that this is actually a negative sentiment because the word ‘love’ by itself is going to be highly correlated with a positive review.

The scikit-learn library makes this really easy to play around with. Just use the ngram_range argument with any of the ‘Vectorizer’ classes.

In [44]:
from nltk import ngrams

sentence = 'this is too far'

two = ngrams(sentence.split(), 2)
three = ngrams(sentence.split(), 3)

for grams in two:
  print(grams)
  
print('###############')

for grams in three:
  print(grams)

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3))

vector = ngram_vectorizer.fit_transform([sentence]).toarray()
print(vector)
print(len(vector[0]))

('this', 'is')
('is', 'too')
('too', 'far')
###############
('this', 'is', 'too')
('is', 'too', 'far')
[[1 1 1 1 1 1 1 1 1]]
9


In [45]:
ngram_vectorizer = CountVectorizer(binary=True, stop_words=english_stop_words, ngram_range=(1, 2))

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model(X, X_test)

Final accuracy: 0.88896


In [46]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 1865232)
Diff X normal y X tras lematizador y vectorización: -1778169


# TF-IDF

Another common way to represent each document in a corpus is to use the tf-idf statistic (term frequency-inverse document frequency) for each word, which is a weighting factor that we can use in place of binary or word count representations.

There are several ways to do tf-idf transformation but in a nutshell, **tf-idf aims to represent the number of times a given word appears in a document (a movie review in our case) relative to the number of documents in the corpus that the word appears in** — where words that appear in many documents have a value closer to zero and words that appear in less documents have values closer to 1.

**Note:** Now that we’ve gone over n-grams, when I refer to ‘words’ I really mean any n-gram (sequence of words) if the model is using an n greater than one.

In [49]:
'''
1 + log((N + 1) / (df + 1))
'''

sent1 = 'My name is Ralph'
sent2 = 'Ralph is fat'
sent3 = 'Ralph'

from sklearn.feature_extraction.text import TfidfVectorizer

test = TfidfVectorizer()
test.fit_transform([sent1, sent2, sent3])
print(test.idf_)
print(test.get_feature_names())

[1.69314718 1.28768207 1.69314718 1.69314718 1.        ]
['fat', 'is', 'my', 'name', 'ralph']


In [50]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)


train_model(X, X_test)

Final accuracy: 0.88376


# Support Vector Machines (SVM)

Recall that linear classifiers tend to work well on very sparse datasets (like the one we have). Another algorithm that can produce great results with a quick training time are Support Vector Machines with a linear kernel.

Build a model with an n-gram range from 1 to 2:

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# SVM con bigramas
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)



def train_model_svm(X_TRAIN, X_TEST):
    svm = LinearSVC()

    params = {
        'C': [0.01, 0.05, 1.25, 0.5, 1]
    }

    grid = GridSearchCV(svm, params, cv=5)
    grid.fit(X_TRAIN, target)

    print('Final accuracy: %s' % accuracy_score(target, grid.best_estimator_.predict(X_TEST)))
    

train_model_svm(X, X_test)

Final accuracy: 0.89768


# Final Model

Removing a small set of stop words along with an n-gram range from 1 to 3 and a linear support vector classifier shows the best results.

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)

ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model_svm(X, X_test)

Final accuracy: 0.90048


# Top Postitive and Negative Features

Obtain the most important features of the model.

In [53]:
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
log_reg = LogisticRegression(C=0.5)
log_reg.fit(X, target)

# Montamos un diccionario con palabra -> coeficiente
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), log_reg.coef_[0]
    )
}

In [56]:
for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(best_positive)

print()
print('###########################################')
print()

for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1])[:10]:
    print(best_negative)

('excellent', 1.379601505780561)
('refreshing', 1.2806374216124707)
('perfect', 1.2012612007522496)
('superb', 1.1357241660871282)
('appreciated', 1.1339191154140513)
('wonderfully', 1.1188417932505035)
('rare', 1.0905178673184064)
('surprisingly', 1.067388747419645)
('incredible', 1.0458950454791647)
('enjoyable', 1.0393834600447407)

###########################################

('worst', -2.0614969453813132)
('waste', -1.9184173170235908)
('disappointment', -1.6761904768996008)
('poorly', -1.6579932684784908)
('awful', -1.5348355500113027)
('disappointing', -1.362106053810215)
('boring', -1.3339611335470245)
('lacks', -1.3127009529582725)
('unfunny', -1.2747882420963395)
('laughable', -1.2741563288429574)


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

print(model)

y_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
